In [353]:
import pandas as pd
import numpy as np

In [354]:
# Import tabula to read pdf file
import tabula

In [ ]:
mak23 = tabula.read_pdf("examples/Mak2023-2024.pdf", pages="all")

In [ ]:
# This data is rough and improperly structured
mak23

In [ ]:
# Index the first table in the mak23 pdf
# Data is highly unordered and requires pre-processing
# A number of techniques shall be used to clean and prepare data into proper structured
# tabular format.
mak23[0]

In [ ]:
# Drop missing data rows.
# It can be observed that index 3 contains what would have been column names: Form id, Index_no, ...
# It also contains original data index read as column: Unnamed: 0
data1 = mak23[0].dropna()

In [ ]:
data1

In [ ]:
# This returns the columns in the DataFrame
data1.columns

In [ ]:
# drop column: Unnamed: 0
del data1["Unnamed: 0"]

In [ ]:
len(data1['MAKERERE UIVERSITY'])

In [ ]:
# Key data is in column Makerere University
# Employ python string processing methods to extracts such data

In [ ]:
data1.iloc[0]

In [ ]:
data1

In [ ]:
data1.index

In [ ]:
# Reindex the dataframe
data2 = data1.reindex([x for x in range(len(data1.index))])

In [ ]:
data2

In [ ]:
newIndex = [x for x in range(len(data1.index))]
type(newIndex)

In [ ]:
data1

In [ ]:
# A series of the total weights
weights = data1["Unnamed: 2"][1:]
weights

In [ ]:
# A series of form IDs
formId = data1['Unnamed: 1'][1:]
formId

In [ ]:
# Preprocess and extract migled data from 'MAKERERE UIVERSITY' column
data2 = data1['MAKERERE UIVERSITY']

In [ ]:
# data[1]
data = [x for x in data2[1:]]
data

In [ ]:
# Rewrting index,
DataColumns = ["Index No", "NAME", "GENDER", "UACE YEAR", "CODE", "DISTRICT", "COURSE CODE", "COURSE NAME"]

In [ ]:
# Function to extract data from data2
def dataProcess(n):
#     if n >= 1:
    studentData = data[n].split()
    index_no = studentData[0]
    name = " ".join(studentData[1:-7])[:-1]
    gender = studentData[-8][-1]
    UACE_YEAR = int(studentData[-7])
    DistrictCode = int(studentData[-6])
    District = studentData[-5]
    CourseCode = studentData[-4]
    Course = ' '.join(studentData[-3:])
    return [index_no, name, gender, UACE_YEAR,
            DistrictCode, District, CourseCode, Course]

In [ ]:
# Create a data frame with no data
df = pd.DataFrame([], columns=DataColumns)

In [ ]:
df

In [ ]:
data

In [ ]:
studentList = []
for i in range(len(data)):
#     if i > 0:
    student = dataProcess(i)
    studentList.append(student)

In [ ]:
studentList

In [ ]:
df = pd.DataFrame(studentList, columns=DataColumns)

In [ ]:
df

In [ ]:
# Insert Total Weight Column into the DataFrame
df.insert(8, "Total Weight", pd.Series([x for x in weights], index=df.index))

In [ ]:
# Insert formId Column as the first column
# Qn: Can we insert multiple columns at once🤔
df.insert(0, "Form ID", pd.Series([x for x in formId], index=df.index))

In [ ]:
df

<h3>Assignment: Preprocess the remaining data tables to have them similar to the data above.</h3>

In [ ]:
# Are there better ways of doing the above tasks?

# <h3>Data description</h3>

In [ ]:
# which district had a higher intake and what is its frequency
df.DISTRICT.describe()

In [ ]:
# In the data portion, there are 26 unique districts.
# Wakiso has oneof the highest frequency of student admissions.
# Budaka is one of the districts with the lowest frequency admissions.

In [ ]:
# why is Budaka returned as the district with the lowest frequency
df.DISTRICT.min()

In [ ]:
# why is Wakiso returned as the district with the highest frequency
# df.DISTRICT.max(include='all')

In [ ]:
df.DISTRICT.value_counts()

In [ ]:
# analysis by SEX

In [ ]:
# More males than females were admitted
df.GENDER.value_counts()

In [ ]:
# Count gender by district 
df_district_gender = pd.DataFrame(data = df.GENDER.value_counts(), index = list(df.DISTRICT), columns = list(df.GENDER))

In [ ]:
# How can we get district gender count?

In [ ]:
# Extract one record
record0 = df.iloc[0]

In [ ]:
# Extracting multiple column data
record0[["Form ID", "NAME"]]

In [ ]:
record0

In [ ]:
record1 = df.iloc[1]

In [ ]:
record1

In [ ]:
# you can concatenate records from different dataframes (sources)
# For example, in the admission data, you what to get a dataframe for students from a specific district.
newdf = pd.concat([record0, record1], keys=df.index).unstack()

In [ ]:
newdf

In [ ]:
# This is an extract of students from Wakiso
wakisoStudents = df.loc[df["DISTRICT"] == "WAKISO"]

In [ ]:
# function definition
def district(district):
    return df.loc[df["DISTRICT"] == district].GENDER.value_counts()

In [ ]:
district("WAKISO")

In [ ]:
wakisoStudents

In [ ]:
# Value Count: Returns count of distinct values
wakisoStudents["GENDER"].value_counts()

In [ ]:
districts = pd.Series(df.DISTRICT)

In [ ]:
districts

In [ ]:
distDict = {}
for i in districts:
#     print(i, district(i))
    distDict[i] = dict(district(i))

In [ ]:
# by default, unstack works for the inner index in hierarchical index.
distResult = pd.DataFrame(distDict).unstack()

In [ ]:
# distResult

In [ ]:
distGender = pd.DataFrame(distResult)

In [ ]:
# distGender.columns.names = ["Enrollment"]

In [ ]:
distGender.index.names = ["District", "Gender"]

In [ ]:
# Renaming column to count
distGender = distGender.rename(columns = {0: "Count"})

In [ ]:
distGender = distGender.fillna(0)

In [ ]:
# Indexing the first two rows
distGender[:3]

In [ ]:
# Indexing a specific district
distGender.loc["IGANGA"]

In [ ]:
district("MASAKA")

In [ ]:
wakisoStudents

In [ ]:
kampalaAdmissions = df[df.DISTRICT == 'KAMPALA']

In [ ]:
kampalaAdmissions

In [ ]:
# Getting Admission numbers by Region: Central, Eastern, Western, Northern
# Some of the representative districts in different regions are;
central = {'Kampala', 'Wakiso', 'Mpigi', 'Mukono', 'Luwero'}

In [ ]:
eastern = {'Iganga', 'Mayuge', 'Jinja', 'Bugiri'}

In [ ]:
western = {'Mbarara', 'Rukugiri', 'Kabale', 'Kisoro'}

In [ ]:
northern = {'Gulu', 'Amolatar', 'Adjumani', 'Apac', 'Arua', 'Kaabong', 'Kitgum'}

In [ ]:
mak23[1]

# Combining more tables

In [ ]:
studentData2 = mak23[1]

In [ ]:
# The df columns need editing
studentData2

In [ ]:
# delete "41" column
del studentData2['41']

In [ ]:
xdata = dataTable2.columns

In [ ]:
# Change columns by re-assigning
DataColumns = df.columns
DataColumns

In [ ]:
# dataframe with the first entry as xdata
df2 = pd.DataFrame([xdata], columns=DataColumns)

In [ ]:
df2

In [ ]:
studentData2.columns = DataColumns

In [ ]:
df2 = pd.concat([df2, studentData2], ignore_index=True)

In [ ]:
df2

In [ ]:
# concatenate d2 table and df.
pd.concat([df, df2], ignore_index=True)